In [1]:
from transformers import *
import os
import pandas as pd
import tokenizers

I0515 17:39:17.986014 139928063416064 file_utils.py:41] PyTorch version 1.4.0 available.
I0515 17:39:19.129147 139928063416064 file_utils.py:57] TensorFlow version 2.1.0 available.


In [2]:
# model_type = "roberta-base"

# tokenizer =  RobertaTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
#             merges_file="transformers_vocab/{}-merges.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )

model_type = "albert-base-v2"

tokenizer = AlbertTokenizer.from_pretrained(
            pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
            lowercase=True,
        )

def get_prediction_v3(tokenizer, tweet, selected_text):
    
    tweet_copy = " ".join(str(tweet).split())
    tweet = " ".join(str(tweet).split()).replace("`", "'")
    selected_text = " ".join(str(selected_text).split()).replace("`", "'")
    
    tweet_copy = " " + tweet_copy.lower()
    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets = []
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for word in tweet.split():
        
        sub_words = tokenizer.tokenize(" " + word)
        encoded_word = tokenizer.convert_tokens_to_ids(sub_words)
#         print(" " + word, tokenizer.convert_ids_to_tokens(encoded_word))
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word
        
        start_offsets = cursor
        cursor += len(" " + word)
        end_offsets = cursor
        
        token_level_cursor = start_offsets
        
        for i in range(number_of_tokens):
            
            tweet_offsets.append((start_offsets, end_offsets))
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + len(sub_words[i])))
            token_level_cursor += len(sub_words[i])
        
#     print(tweet_offsets)
#     print(tweet_offsets_token_level)
    
    # get word idx
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
#         print(tweet[offset1: offset2])
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     print(targets_start, targets_end)
    
    
    return tweet_copy[tweet_offsets[targets_start][0]: tweet_offsets[targets_end][1]], \
            tweet_copy[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]]

# tweet = "star wars ............ is **** boo??? i wanna do your job hand it over u can act as me at my high school lol"
# selected_text = "l"

tweet = "i`m soooooo sleeeeepy!!!"
selected_text = "soooooo sleeeeepy!!!"

get_prediction_v3(tokenizer, tweet, selected_text)

I0515 17:39:19.362255 139928063416064 tokenization_utils.py:420] Model name 'transformers_vocab/albert-base-v2-spiece.model' not found in model shortcut name list (albert-base-v1, albert-large-v1, albert-xlarge-v1, albert-xxlarge-v1, albert-base-v2, albert-large-v2, albert-xlarge-v2, albert-xxlarge-v2). Assuming 'transformers_vocab/albert-base-v2-spiece.model' is a path, a model identifier, or url to a directory containing tokenizer files.
W0515 17:39:19.363045 139928063416064 tokenization_utils.py:432] Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
I0515 17:39:19.363567 139928063416064 tokenization_utils.py:502] loading file transformers_vocab/albert-base-v2-spiece.model


(' soooooo sleeeeepy!!!', ' soooooo sleeeeepy!!!')

In [3]:
model_type = "roberta-base"

tokenizer_0 = RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tokenizer_1 = tokenizers.ByteLevelBPETokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tweet = " SEe waT I Mean bOuT FoLL0w fRiiDaYs... It`S cALLed LoSe f0LloWeRs FridAy... smH"


input_ids_orig_0 = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(tweet))

ids_0 = []
offsets_0 = []
offsets_0_token_level = []
cursor = 0


for i, word in enumerate(tweet.lower().split()):
    
    encoded_word = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(" " + word))
    ids_0 += encoded_word
    
    number_of_tokens = len(encoded_word)

    start_offsets = cursor
    cursor += len(" " + word)
    end_offsets = cursor
    
    token_level_cursor = start_offsets

    for i in range(number_of_tokens):
        offsets_0.append((start_offsets, end_offsets))
        curr_token = tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(encoded_word[i]))
        
        offsets_0_token_level.append((token_level_cursor, token_level_cursor + len(curr_token)))
        token_level_cursor += len(curr_token)
    
# print(input_ids_orig_0)
# print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(input_ids_orig_0)))
print(ids_0)
print(offsets_0_token_level)
print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(ids_0)))

ids_1 = tokenizer_1.encode(tweet).ids
print(ids_1)
print(tokenizer_1.encode(tweet).offsets)
print(tokenizer_1.decode(ids_1))

[192, 20162, 939, 1266, 9953, 40629, 288, 605, 40404, 32904, 734, 24, 12905, 29, 373, 2217, 856, 288, 890, 9945, 6664, 21746, 734, 5278, 298]
[(0, 4), (4, 8), (8, 10), (10, 15), (15, 20), (20, 25), (25, 26), (26, 27), (27, 31), (31, 36), (36, 39), (39, 42), (42, 43), (43, 44), (44, 51), (51, 56), (56, 58), (58, 59), (59, 61), (61, 66), (66, 69), (69, 73), (73, 76), (76, 79), (79, 80)]
 see wat i mean bout foll0w friidays... it`s called lose f0llowers friday... smh
[192, 20162, 939, 1266, 9953, 40629, 288, 605, 40404, 32904, 734, 24, 12905, 29, 373, 2217, 856, 288, 890, 9945, 6664, 21746, 734, 5278, 298]
[(0, 4), (4, 8), (8, 10), (10, 15), (15, 20), (20, 25), (25, 26), (26, 27), (27, 31), (31, 36), (36, 39), (39, 42), (42, 43), (43, 44), (44, 51), (51, 56), (56, 58), (58, 59), (59, 61), (61, 66), (66, 69), (69, 73), (73, 76), (76, 79), (79, 80)]
 see wat i mean bout foll0w friidays... it`s called lose f0llowers friday... smh


In [4]:
train = pd.read_csv("/media/jionie/my_disk/Kaggle/Tweet/input/tweet-sentiment-extraction/train.csv")

In [6]:
model_type = "albert-base-v2"

tokenizer = AlbertTokenizer.from_pretrained(
            pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
            lowercase=True,
        )

for i in range(len(train)):
    
    label = str(train["selected_text"].iloc[i]).lower().strip()
    prediction, prediction_token = get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i])
    prediction = prediction.strip()
    prediction_token = prediction_token.strip()
    print(prediction, "--------------------", prediction_token)
#     if label != prediction:
#         print(
#               label, \
#               "-------------------", \
#               get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i]), \
#               "-------------------", train["text"].iloc[i])

I0515 17:39:54.595472 139928063416064 tokenization_utils.py:420] Model name 'transformers_vocab/albert-base-v2-spiece.model' not found in model shortcut name list (albert-base-v1, albert-large-v1, albert-xlarge-v1, albert-xxlarge-v1, albert-base-v2, albert-large-v2, albert-xlarge-v2, albert-xxlarge-v2). Assuming 'transformers_vocab/albert-base-v2-spiece.model' is a path, a model identifier, or url to a directory containing tokenizer files.
W0515 17:39:54.596491 139928063416064 tokenization_utils.py:432] Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
I0515 17:39:54.596967 139928063416064 tokenization_utils.py:502] loading file transformers_vocab/albert-base-v2-spiece.model


i`d have responded, if i were going -------------------- i`d have responded, if i were going
sooo sad -------------------- sooo sad
bullying me... -------------------- bullying me
leave me alone -------------------- leave me alone
sons of ****, -------------------- sons of ****,
http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth -------------------- http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth
fun -------------------- fun
soooo high -------------------- soooo high
both of you -------------------- both of you
wow... u just became cooler. -------------------- wow... u just became cooler.
as much as i love to be hopeful, i reckon the chances are minimal =p i`m never gonna get my cake and stuff -------------------- as much as i love to be hopeful, i reckon the chances are minimal =p i`m never gonna get my cake and stuff
like -------------------- like
dangerously -------------------- dangerously
lost -

that`s not cool -------------------- that`s not cool
alwas helps. -------------------- alwas helps.
lacking, -------------------- lacking,
cool -------------------- cool
good -------------------- good
i think it`s under a honeymoon by the good life -- either that or it`s under a honey moon by joseph arthur. -------------------- i think it`s under a honeymoon by the good life -- either that or it`s under a honey moon by joseph arthur.
swollen. -------------------- swollen.
it was hours ago i came in.... and it`s only now i realised when i went to buy something online -------------------- it was hours ago i came in.... and it`s only now i realised when i went to buy something online
oh just referring to our lil exchange on lj with regards to twitter archive postings. -------------------- oh just referring to our lil exchange on lj with regards to twitter archive postings.
don`t act like u don`t know. -------------------- don`t act like u don`t know.
best -------------------- best
role! i

tiiiiiiired! going to bed! -------------------- tiiiiiiired! going to bed!
just been in that kind of mood, not reason at all. lol, but i`ll try not to be too mushy around you! i can behave! -------------------- just been in that kind of mood, not reason at all. lol, but i`ll try not to be too mushy around you! i can behave!
i hope -------------------- i hope
keeps getting such delayed responses. why is my internet so messed up?! -------------------- keeps getting such delayed responses. why is my internet so messed up?!
and dont tell burnsy - but no comparison between the rocky mountains and mountains in enlgland -------------------- and dont tell burnsy - but no comparison between the rocky mountains and mountains in enlgland
sad. -------------------- sad.
good -------------------- good
i could get away with it. dare me? i`ll go buy it tomorrow -------------------- i could get away with it. dare me? i`ll go buy it tomorrow
thanks man. that sorted it. -------------------- thanks man. t

win -------------------- win
live long -------------------- live long
just got in from michaels and had some shreddies im well tired, it was fun though ryan wright stoned = lol -------------------- just got in from michaels and had some shreddies im well tired, it was fun though ryan wright stoned = lol
hey charicee! how are u? are you going to come to italy again? please tell me yes -------------------- hey charicee! how are u? are you going to come to italy again? please tell me yes
mean -------------------- mean
finished proofing the wedding from march .. now just waiting for export and upload before i can share still have 2 more sessions to do! -------------------- finished proofing the wedding from march .. now just waiting for export and upload before i can share still have 2 more sessions to do!
lol yea im thinkin bday bash ! but everybodys lame wants to sit in the house instead of qo wit me lol -------------------- lol yea im thinkin bday bash ! but everybodys lame wants to sit

...

elementary -------------------- elementary
so sad. -------------------- so sad.
was in pain -------------------- was in pain
sitting outside in the cold by the ocean with a glass of wine. -------------------- sitting outside in the cold by the ocean with a glass of wine.
i got luck for u guys to beat la!! -------------------- i got luck for u guys to beat la!!
i missed -------------------- i missed
stupid sun -------------------- stupid sun
wtf -------------------- wtf
happy mother`s day to every single mom out there. i love my mommy -------------------- happy mother`s day to every single mom out there. i love my mommy
depressed -------------------- depressed
just realized i have 30 followers hey guys -------------------- just realized i have 30 followers hey guys
not to worry, -------------------- not to worry,
twitter is trippin right now with my pic -------------------- twitter is trippin right now with my pic
the weather is perfect for it -------------------- the weather is perfect

my brain is gonna explode, a minute from now ) -------------------- my brain is gonna explode, a minute from now )
bad -------------------- bad
had bad -------------------- had bad
_c i can`t find the original on blip.fm and the ex models version, not so much... -------------------- _c i can`t find the original on blip.fm and the ex models version, not so much...
fantastic.... -------------------- fantastic.
basil will be the highlight of my day. i`ve just had an old man lecture me on the greatness of the telegraph. what an idiot -------------------- basil will be the highlight of my day. i`ve just had an old man lecture me on the greatness of the telegraph. what an idiot
happy -------------------- happy
bored. -------------------- bored.
soon!! love -------------------- !! love
please -------------------- please
wow justin timberlake on snl tonight was hilarious! you gotta love that dude! -------------------- wow justin timberlake on snl tonight was hilarious! you gotta love that dude

it was fun it`s been nine months! -------------------- it was fun it`s been nine months!
i thought there was more fanta in my can than there was -------------------- i thought there was more fanta in my can than there was
excited -------------------- excited
watching rugrats anndd tweeting, of course! -------------------- watching rugrats anndd tweeting, of course!
glad you`re alright! -------------------- glad you`re alright!
good luck chan! -------------------- good luck chan!
don`t want to go!!! -------------------- don`t want to go!!!
its not it just sucks -------------------- its not it just sucks
guitar lessons tomorrow. ( i have to wake up early again. -------------------- guitar lessons tomorrow. ( i have to wake up early again.
nope, thank god, not on air... it`s a real application, compiled code... -------------------- nope, thank god, not on air... it`s a real application, compiled code...
addictive isn`****. biz could definately be better! folks have just slowed down on get

it`s nice to see you! -------------------- it`s nice to see you!
wish -------------------- wish
not fair. -------------------- not fair.
thanks, -------------------- thanks,
sorry -------------------- sorry
being slammed with -------------------- being slammed with
happy -------------------- happy
i love -------------------- i love
ok... maybe not angry... just with very little sense -------------------- ok... maybe not angry... just with very little sense
excited to see 'up' for logan`s birthday in 2 weeks. sad he wont have a party bc he doesn`t know any kids, tho. -------------------- excited to see 'up' for logan`s birthday in 2 weeks. sad he wont have a party bc he doesn`t know any kids, tho.
sux u will be poor tho -------------------- sux u will be poor tho
that`s **** awesome! -------------------- that`s **** awesome!
excellent, he will never know, lmao. -------------------- excellent, he will never know, lmao.
i was too much of a punk **** -------------------- i was too much of 

happy mother`s day -------------------- happy mother`s day
is tired! -------------------- is tired!
tv time see yall later! xoxo -------------------- tv time see yall later! xoxo
came from greenbelt & power -------------------- came from greenbelt & power
that sucks -------------------- that sucks
got any tips for a mum of a 14 yr old with cfs who can`t sleep at night. he falls asleep about 4a.m. till midday or so -------------------- got any tips for a mum of a 14 yr old with cfs who can`t sleep at night. he falls asleep about 4a.m. till midday or so
i`m sorry! lol i have never had, like, actual time to spend there unfortunately. everything`s always been insane when i go -------------------- i`m sorry! lol i have never had, like, actual time to spend there unfortunately. everything`s always been insane when i go
wweeeeooo! snl soon -------------------- wweeeeooo! snl soon
awesome! -------------------- awesome!
<3 to my favorite blogging site. -------------------- <3 to my favorite blo

In [ ]:
tokenizer.tokenize(tweet)

In [ ]:
encoded = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet))
encoded

In [ ]:
decoded = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded))
decoded

In [ ]:
print(len(encoded), len(tokenizer.tokenize(tweet)))

In [ ]:
tweet.split()

In [ ]:
for (i, word) in enumerate(tweet.split()):
        
    sub_tokens = tokenizer.tokenize(word)

    print(sub_tokens)